In [24]:
import tensorflow as tf
from keras.applications.mobilenet_v2 import MobileNetV2

from sklearn.model_selection import train_test_split
from pathlib import Path
import matplotlib.image as img
import json
import numpy as np

In [4]:
# from tensorflow.python.client import device_lib
# device_lib.list_local_devices()

In [5]:
# Importing Data
# Notes:

# Orientation
# 1: portrait
# 2: portrait, upside down (iPad only)
# 3: landscape, with home button on the right
# 4: landscape, with home button on the left

In [9]:
# Helper Functions from EDA
def ejson(p, fn): # extract json
    with open((p/fn).as_posix()) as f: return json.load(f) 
## Simple Json Reading Functions
def info_data(p): return ejson(p, 'info.json')
def dot_data(p): return ejson(p, 'dotInfo.json')
def frame_data(p): return ejson(p, 'frames.json')    
def screen_data(p): return ejson(p, 'screen.json')
def get_frame(p, img_fn): return img.imread(p/'frames'/img_fn)
## Larger Helper Functions
def coordinate_data(p): 
    data = dot_data(p)
    return data['XCam'], data['YCam'] # we want relative to camera coords

In [4]:
# dset_path = Path('ml/gazecapture/')
dset_path = Path('../gazecapture/')

In [29]:
X = []

Y_Xpoint, Y_Ypoint = [], []

for case in dset_path.iterdir():
    XCam, YCam = coordinate_data(case) # list (x, y) coordinates rel to camera 
    F = frame_data(case) # list of file names correlated to camera images 
#     O = screen_data(case)['Orientation']
    dset_class = info_data(case)['Dataset'] # used for splitting into test / train / val
    
    for i in range(len(F)): # iterate each x, y value
        x = get_frame(case, F[i]) # get the photo as matrix
        xpoint, ypoint = XCam[i], YCam[i] # coordinate to predict 
        
        X.append(x)
        Y_Xpoint.append(xpoint)
        Y_Ypoint.append(ypoint)
#         o = O[i] 
    break

X = np.asarray(X)
Y_Xpoint, Y_Ypoint = np.asarray(Y_Xpoint), np.asarray(Y_Ypoint)

In [30]:
# training and validation split
x_train, x_val, y_xpoint_train, y_xpoint_val, y_ypoint_train, y_ypoint_val \
    = train_test_split(X, Y_Xpoint, Y_Ypoint, test_size=0.2, random_state=1)

In [33]:
x_train.shape, y_xpoint_train.shape, y_ypoint_train.shape

((81, 640, 480, 3), (81,), (81,))